In [1]:
from pytube import YouTube
import torch
from utils.general import (check_file, check_img_size, cv2, non_max_suppression)

/home/theo/anaconda3/envs/flaskenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
yt = YouTube('https://www.youtube.com/watch?v=WfuE61ZkuRo')
video = yt.streams.filter(file_extension='mp4').first()
video.download('video/')

In [3]:
from pytube import YouTube
def pytube_dl(url):
    yt = YouTube(str(url))
    video = yt.streams.filter(file_extension='mp4').first()
    video.download('video/')


In [2]:
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages

Using cache found in /home/theo/.var/app/com.visualstudio.code/cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-7-5 Python-3.10.4 torch-1.12.0+cu102 CPU

Fusing layers... 


requirements: /home/theo/Desktop/apiBrandseeker/requirements.txt not found, check failed.


Model summary: 213 layers, 7112611 parameters, 0 gradients
Adding AutoShape... 


In [9]:
from tqdm.autonotebook import tqdm

In [ ]:
i=0
for path, im, im0s, vid_cap, s, frame in tqdm(dataset, total=total_frames):
    i+=1
    print(i)


In [10]:
from utils.torch_utils import select_device, time_sync

YOLOv5 🚀 2022-7-5 Python-3.10.4 torch-1.12.0+cu102 CPU



In [11]:
from utils.filtering import filter_output
from utils.pdf_generator import pdf_generator, normalize

YOLOv5 🚀 2022-7-5 Python-3.10.4 torch-1.12.0+cu102 CPU



In [16]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size((640, 640), s=stride)  

dataset = LoadImages("video/", img_size=imgsz, stride=stride, auto=pt, only_vids=True)
bs = 1
brand_count = {}
path, im, im0s, vid_cap, s, frame = dataset.__iter__().__next__()

framerate = 1
initial_framerate = vid_cap.get(cv2.CAP_PROP_FPS)
real_framerate = initial_framerate / round(initial_framerate / framerate)
total_frames = dataset.frames
dataset.frame = 0

total_frames = dataset.frames
device = select_device('')
dt, seen = [0.0, 0.0, 0.0], 0

save_dir = "saving_predict"
save_unprocessed_output = False
pred_timing_start = time_sync()

for path, im, im0s, vid_cap, s, frame in tqdm(dataset, total=total_frames):

        # skip the frame if it isn't in the specified framerate
        if frame % round(initial_framerate / framerate) != 0:
            continue

        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.float()
        im /= 255  # 0 - 255 to 0.0 - 1.0

        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        pred = model(im)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres=0.35, max_det=5)
        pred = pred[0].tolist()
        dt[2] += time_sync() - t3

        has_prediction = len(pred)
        if has_prediction:
            for brand in pred:
                label = names[int(brand[5])]

                # Retrieve or create a dictionnary key for the label and add the bbox, confidence and frame of the prediction
                brand_count[label] = brand_count.get(label, {"bbox": [], "confidence": [], "frame": []})
                brand_count[label]["bbox"].append(brand[0:4])
                brand_count[label]["confidence"].append(brand[4])
                brand_count[label]["frame"].append(frame)
    
    # Generate an output if a prediction has been made
if brand_count:
    filtered_output = filter_output(brand_count, framerate)
    pdf_generator(path, filtered_output, save_dir)
    # cv2.imshow(im, framerate)

    if save_unprocessed_output:
        with open(f"{save_dir}/{normalize(path)}.txt", "w") as f:
            f.write(str(brand_count))
else:
    print("No prediction has been made")


pred_timing_stop = time_sync()
pred_timing = pred_timing_stop - pred_timing_start
print("Pred took %.2fs (%.2ffps)" % (pred_timing, ((total_frames / initial_framerate) * real_framerate) / pred_timing))


 96%|█████████▌| 33687/35128 [03:30<00:08, 160.31it/s]


{'Winamax': [1.6746974454223433, 0.7869841456413269, 14730, [279.46917724609375, 143.6875457763672, 421.904052734375, 275.8118896484375]], 'Lootcrate': [0.09001361782811455, 0.7459093630313873, 675, [12.015167236328125, 12.05072021484375, 619.912109375, 345.76708984375]], 'Manscaped': [0.10441062780983588, 0.6591405272483826, 3090, [22.774757385253906, 259.8238525390625, 219.39822387695312, 315.1514892578125]], 'Uber Eats': [0.15801738860836173, 0.39638763666152954, 18570, [578.1520385742188, 57.36253356933594, 608.5293579101562, 72.75448608398438]]}
Pred took 251.39s (69.87fps)
